In [2]:
import pandas as pd

def parse_dialog(sentences):
    return '; '.join(sentences)

def make_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

test_df = make_df('data/test.statement.jsonl',is_test=True)

In [21]:
def parse_request(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"

In [22]:
test_df['req'] = test_df.apply(parse_request,axis=1)

In [5]:
!mkdir ./real_dream
%cd real_dream

!git clone https://github.com/nlpdata/dream.git

/notebooks/real_dream/real_dream
Cloning into 'dream'...
remote: Enumerating objects: 56, done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56 (from 1)
Receiving objects: 100% (56/56), 1.38 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [10]:
%cd ./../

/notebooks


In [16]:
def make_df_test(path):
    data = pd.read_json(path)
    df = []
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels':  ques['answer']
            })
    return pd.DataFrame(df)



In [17]:
test = make_df_test('real_dream/real_dream/dream/data/test.json')
train = make_df_test('real_dream/real_dream/dream/data/train.json')
dev = make_df_test('real_dream/real_dream/dream/data/dev.json')

In [26]:
test['req'] = test.apply(parse_request,axis=1)
train['req'] = train.apply(parse_request,axis=1)
dev['req'] = dev.apply(parse_request,axis=1)

In [36]:
all_df = pd.concat([
    test,train,dev
],axis=0,ignore_index=True)

In [38]:
req_ans_maper = dict(zip(all_df['req'].tolist(),all_df['labels'].tolist()))

In [42]:
test_df['answers'] = test_df['req'].map(req_ans_maper)

In [50]:
test_df.to_csv('dream_test_answers.csv',index=False)